Top 20 Most Active PTT Boards

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.font_manager import FontProperties


# 繁体中文字体配置
def setup_traditional_chinese_font():
    """配置繁体中文字体，确保PTT繁体看板名正常显示"""
    # macOS 繁体字体路径（优先加载）
    mac_font_paths = [
        '/System/Library/Fonts/NewTaipeiSansTC-Regular.ttf',  # 台北思源黑体（首选）
        '/System/Library/Fonts/STHeiti Light.ttc',            # 华文细黑（繁体）
        '/System/Library/Fonts/STSong.ttf',                  # 华文宋体（繁体）
        '/System/Library/Fonts/PingFang.ttc',                # 苹方（兼容繁体）
        '/System/Library/Fonts/Heiti TC.ttc',                # 黑体-繁（备用）
    ]
    
    # 尝试加载系统自带繁体字体
    font_paths = mac_font_paths
    target_font = None
    
    for font_path in font_paths:
        if os.path.exists(font_path):
            try:
                target_font = FontProperties(fname=font_path)
                # 全局设置字体 + 显式指定字体名称（双重保障）
                plt.rcParams['font.family'] = [target_font.get_name()]
                plt.rcParams['axes.unicode_minus'] = False  # 解决负号方块问题
                print(f"成功加载繁体中文字体：{font_path.split('/')[-1]}")
                return target_font
            except Exception as e:
                print(f"加载字体 {font_path} 失败：{e}")
                continue
    
    # 若系统无自带繁体字体，强制使用兼容字体
    fallback_font = FontProperties(family='Arial Unicode MS')
    plt.rcParams['font.sans-serif'] = [fallback_font.get_name()]
    plt.rcParams['axes.unicode_minus'] = False
    print(f"使用兼容字体：Arial Unicode MS（支持繁体中文）")
    return fallback_font

tc_font = setup_traditional_chinese_font()
plt.style.use('default')

def load_and_extract_board(df_path):
    """
    加载数据并从post_title提取PTT看板标签
    返回：清洗后的DataFrame、看板统计结果
    """
    # 1. 加载数据
    df = pd.read_csv(df_path)
    print(f"原始数据总量：{len(df)} 条")
    
    # 2. 提取看板标签（[看板名] 格式）
    # 匹配开头的 [任意字符]（排除特殊符号）
    board_pattern = r'^\[([^\\\[\]]+)\]'  # 匹配 [看板名]，避免匹配嵌套括号
    df['board'] = df['title'].str.extract(board_pattern, expand=False)
    
    # 3. 数据清洗：处理未提取到看板的记录
    df_clean = df[df['board'].notna()].copy()  # 保留有看板标签的记录
    unrecognized = len(df) - len(df_clean)
    print(f"可识别看板的记录数：{len(df_clean)} 条")
    print(f"未识别看板的记录数：{unrecognized} 条（标题格式异常）")
    
    # 4. 统计各看板帖子数量（按数量降序）
    board_post_count = df_clean['board'].value_counts().reset_index()
    board_post_count.columns = ['board_name', 'post_count']
    print(f"\n共识别出 {len(board_post_count)} 个不同看板")
    
    return df_clean, board_post_count


# 可视化：Top N热门看板柱状图
def plot_top_boards_bar(board_post_count, top_n=20):
    """绘制Top N热门看板柱状图（确保繁体中文显示）"""
    # 筛选Top N看板（若总看板数不足N则取全部）
    top_boards = board_post_count.head(top_n)
    plt.figure(figsize=(18, 10))  # 增大图表尺寸，避免标签重叠
    
    # 绘制柱状图
    bars = plt.bar(
        x=range(len(top_boards)),
        height=top_boards['post_count'],
        color='#2E86AB', alpha=0.8, edgecolor='white', linewidth=0.8
    )
    
    # 图表标题和标签
    plt.title(f'Top {len(top_boards)} Most Active PTT Boards (topic about scam)', 
              fontsize=16, fontweight='bold', fontfamily='DejaVu Sans')
    plt.xlabel('Board Name', fontsize=14, fontproperties=tc_font)
    plt.ylabel('Number of Posts', fontsize=14, fontproperties=tc_font)
    
    # X轴标签
    plt.xticks(
        range(len(top_boards)),
        top_boards['board_name'],
        rotation=45,  # 旋转45度，平衡可读性和空间
        ha='right',   # 右对齐，避免标签溢出
        fontsize=11,  # 增大字体，让繁体笔画更清晰
        fontproperties=tc_font  # 强制使用繁体字体
    )
    
    # Y轴刻度标签
    plt.yticks(fontproperties=tc_font)
    
    # 网格线
    plt.grid(axis='y', alpha=0.3, linestyle='--')
    
    # 柱子顶部添加数值标签
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width()/2,
            height + max(top_boards['post_count'])*0.005, 
            f'{int(height)}',
            ha='center', va='bottom', fontweight='bold', fontsize=10, fontproperties=tc_font
        )
    
    # 自动调整布局，防止标签被截断
    plt.tight_layout()
    
    # 保存图表
    save_path = '/Users/apple/Desktop/data/visual_new/top_boards_bar.png'
    plt.savefig(
        save_path,
        dpi=300,
        bbox_inches='tight',
        facecolor='white'  # 白色背景，避免透明导致文字模糊
    )
    plt.close()
    print(f"\nTop看板柱状图已保存至：{save_path}")


# 主执行流程
if __name__ == "__main__":
    df_path = "/Users/apple/Desktop/data/data_clean/ptt_posts_clean(1).csv"
    
    try:
        # 1. 加载数据并提取繁体看板
        df_clean, board_post_count = load_and_extract_board(df_path)
        
        # 2. 生成Top 20看板柱状图（繁体中文正常显示）
        plot_top_boards_bar(board_post_count, top_n=20)
        
        print("\n图表生成完成！请查看桌面 data 文件夹中的 top_boards_bar.png")
    
    except FileNotFoundError:
        print(f"错误：文件未找到，请检查路径 '{df_path}' 是否正确")
    except Exception as e:
        print(f"执行出错：{e}")

成功加载繁体中文字体：STHeiti Light.ttc
原始数据总量：25739 条
可识别看板的记录数：18146 条
未识别看板的记录数：7593 条（标题格式异常）

共识别出 596 个不同看板

Top看板柱状图已保存至：/Users/apple/Desktop/data/visual_new/top_boards_bar.png

图表生成完成！请查看桌面 data 文件夹中的 top_boards_bar.png
